# Codebase Chat Agent

This agent is responsible for answering questions about the codebases. This is a simple ReAct agent with access to a tool to search code snippets.

If the agent need to think about the user query or make function calls, it will do in a tag `<thinking>`. This can used to understand the agent's thinking process. Integrations like OpenWebUI can use this tag to show the agent's thinking process to the user.

## Setup

In [1]:
from rich import print  # noqa: A004

from automation.agents.codebase_chat import CodebaseChatAgent

config = {"run_name": "CodebaseChat"}
codebase_chat = CodebaseChatAgent()

## Graph

In [ ]:
from IPython.display import Image, display  # noqa: A004

display(Image(codebase_chat.draw_mermaid()))

## Examples

Here are some examples of how the agent works. 

### Query unrelated to codebases

The expected behavior is that the agent will answer that it can only answer questions about the codebases or software development in general.

In [ ]:
result = codebase_chat.agent.invoke({"messages": [("human", "Hi, what is the capital of France?")]}, config=config)

for message in result["messages"]:
    print(message.pretty_print())

### Query about software development

The expected behavior is that the agent will answer directly without searching the codebases. 

In [ ]:
result = codebase_chat.agent.invoke({"messages": [("human", "How can I make a good software?")]}, config=config)

for message in result["messages"]:
    print(message.pretty_print())

### Query about a specific repository

The expected behavior is that the agent will search the codebases for the query and answer the question based on the codebases.

In [ ]:
result = codebase_chat.agent.invoke(
    {"messages": [("human", "What is the main programming language used in the codebases?")]}, config=config
)

for message in result["messages"]:
    print(message.pretty_print())

### Query about a specific repository with follow-up questions

It's expected that the agent will answer being aware of previous messages.

In [ ]:
result = codebase_chat.agent.invoke(
    {"messages": [("human", "What's the python version used on django-webhooks?")]}, config=config
)
result = codebase_chat.agent.invoke(
    {"messages": result["messages"] + [("human", "How can i install a new dependency?")]}, config=config
)
for message in result["messages"]:
    print(message.pretty_print())